In [ ]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro, pearsonr, spearmanr
from tqdm import tqdm
import statsmodels.api as sm
from scipy.stats import levene
from statsmodels.tools import add_constant
from statsmodels.stats.multitest import multipletests
from statsmodels.formula.api import ols
import data_preproc as dp
import regress_out_cells as roc

In [ ]:
def levene_preproc(betas, pheno):
    X_r = roc.regress_out_cells(betas, pheno, False)
    df_Xr = pd.DataFrame(X_r, columns = betas.columns)
    df_Xr = add_constant(df_Xr)
    df_Xr = df_Xr.reset_index()
    return df_Xr

def bonferroni_correction(df_result, alpha, function):
    pvalue_adjusted = multipletests(df_result[function + '_p-value'], alpha = alpha, method = 'bonferroni', is_sorted = True)
    col_name = function + '_pvalue-adjusted' 
    df_result[col_name] = pvalue_adjusted[1]
    if function == 'levene':
        df_result = df_result.loc[(df_result[col_name] >= alpha)]
    else:
        df_result = df_result.loc[(df_result[col_name] <= alpha)]
    return df_result

def cpgs_list(cpgs_lst, file_name):
    cpgs_list = open('{0}/{1}_cpgs.txt'.format(directory_folder, file_name), 'w')
    for cpg in cpgs_lst:
        cpgs_list.write(cpg + '\n')
    cpgs_list.close()
    return cpgs_list

In [ ]:
directory_folder = ''
pheno_folder = ''
betas_folder = ''
alpha = 0.001

pheno_central, betas_central, pheno_yakutia, betas_yakutia = dp.data_preproc(directory_folder, pheno_folder, betas_folder)

In [ ]:
betas_central_const = levene_preproc(betas_central, pheno_central)
betas_yakutia_const = levene_preproc(betas_yakutia, pheno_yakutia)

levene_result = {}
for cpg_name in tqdm(betas_central.columns, ncols = 100):
    formula1 = '{0} ~ index + const'.format(cpg_name) 
    formula2 = '{0} ~ index'.format(cpg_name) 
    model1 = ols(formula1, betas_central_const).fit()
    model2 = ols(formula2, betas_yakutia_const).fit()
    lvn = levene(model1.resid, model2.resid)
    levene_result[cpg_name] = lvn[0], lvn[1]
df_result = pd.DataFrame.from_dict(levene_result, orient = 'index', columns = ['levene_statistic', 'levene_p-value'])
df_result = bonferroni_correction(df_result, alpha, 'levene')

In [ ]:
betas_central_const['c'] = 1 
betas_all = pd.concat([betas_central_const, betas_yakutia_const], ignore_index = True).drop('const', axis = 1)
betas_all = betas_all.fillna(0) 
betas_all['int'] = betas_all['index'] * betas_all['c']
for cpg_name in tqdm(df_result.index, ncols = 100):    
    formula3 = '{0} ~ index + c + int'.format(cpg_name) 
    result = ols(formula3, betas_all).fit() 
    hypotheses = '(c = 0), (int = 0)'
    f_test = result.f_test(hypotheses)
    
    df_result.at[cpg_name, 'F-test_statistic'] = f_test.fvalue
    df_result.at[cpg_name, 'F-test_p-value'] = f_test.pvalue

df_result = bonferroni_correction(df_result, alpha, 'F-test')
df_result = df_result.sort_values(by = 'F-test_pvalue-adjusted', ascending = True)

In [ ]:
correlation_central_fdr = pd.read_csv('', sep = '\t')
correlation_yakutia_fdr = pd.read_csv('', sep = '\t')

for i in range(0, 10):
    cpg_name = df_result.index[i]
    fig = plt.figure(figsize = (6, 6))
    plt.plot(pheno_central['Age'], betas_central[cpg_name], 'o', color = '#4DAF4A', label = 'Central')
    plt.plot(pheno_yakutia['Age'], betas_yakutia[cpg_name], 'o', color = '#3366CC', label = 'Yakutia')
    plt.legend()
    fig.patch.set_facecolor('white')

    m, b = np.polyfit(pheno_central['Age'], betas_central[cpg_name], 1)
    plt.plot(pheno_central['Age'], m * pheno_central['Age'] + b, color = '#4DAF4A')
    a, c = np.polyfit(pheno_yakutia['Age'], betas_yakutia[cpg_name], 1)
    plt.plot(pheno_yakutia['Age'], a * pheno_yakutia['Age'] + c, color = '#3366CC')
    
    plt.xlim([10, 105])
    plt.ylim([0, 1])
    plt.title('Corr. coef.: Central: {0:0.3f}, Yakutia: {1:0.3f},\np-value: Central: {2:e}, Yakutia: {3:e}'.format(
        list(correlation_central_fdr['correlation'])[i], list(correlation_yakutia_fdr['correlation'])[i], 
        list(correlation_central_fdr['pvalue-corrected'])[i], list(correlation_yakutia_fdr['pvalue-corrected'])[i]), fontsize = 14)
    plt.ylabel(str(cpg_name))
    plt.xlabel('Age')
    fig.tight_layout()
    plt.savefig('{0}/{1}_central.png'.format(directory_folder, cpg_name), dpi = 300)
    plt.close()

In [ ]:
ftest_unique = np.setdiff1d(df_result.index, list(correlation_central_fdr['cpg']))
print('Unique CpGs selected by F-test analysis:', len(ftest_unique))

correlation_unique = np.setdiff1d(list(correlation_central_fdr['cpg']), df_result.index)
print('Unique CpGs selected by correlation analysis:', len(correlation_unique))

general_correlation_ftest = list(set(correlation_central_fdr['cpg']) & set(df_result.index))
print('General CpGs:', len(general_correction_ftest))

ftest_all_cpgs = cpgs_list(df_result.index, 'ftest_all')
ftest_unique_cpgs = cpgs_list(ftest_unique, 'ftest_unique')
correlation_unique_cpgs = cpgs_list(correlation_unique, 'correlation_unique')
general_correlation_ftest_cpgs = cpgs_list(general_correction_ftest, 'general_correlation_ftest')

with pd.ExcelWriter('{0}/f-test_results.xlsx'.format(directory_folder)) as writer:
    df_result.to_excel(writer, index = True)